In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Djambala,CG,2021-07-23 21:35:04,-2.5447,14.7533,73.44,48,75,5.57
1,1,Castro,BR,2021-07-23 21:35:04,-24.7911,-50.0119,54.68,77,0,3.31
2,2,Port Shepstone,ZA,2021-07-23 21:35:05,-30.7414,30.4550,53.94,62,51,15.95
3,3,Punta Arenas,CL,2021-07-23 21:35:05,-53.1500,-70.9167,37.51,87,40,17.27
4,4,Tasiilaq,GL,2021-07-23 21:33:22,65.6145,-37.6368,41.16,99,100,4.50


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps,dissipating=False, max_intensity=300,point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
10,10,Guerrero Negro,MX,2021-07-23 21:35:07,27.9769,-114.0611,81.88,54,21,15.79
21,21,Kavaratti,IN,2021-07-23 21:35:09,10.5669,72.6420,81.70,82,93,21.63
27,27,Puerto Ayora,EC,2021-07-23 21:35:11,-0.7393,-90.3518,82.36,82,98,4.00
29,29,Victoria,HK,2021-07-23 21:31:19,22.2855,114.1577,86.38,86,74,1.01
32,32,Guajara,BR,2021-07-23 21:35:13,-2.9667,-57.6667,85.05,53,83,2.48
51,51,Fuerte Olimpo,PY,2021-07-23 21:35:18,-21.0415,-57.8738,81.21,26,0,5.82
54,54,Paracuru,BR,2021-07-23 21:31:02,-3.4100,-39.0306,80.62,77,10,13.89
59,59,Butaritari,KI,2021-07-23 21:34:33,3.0707,172.7902,83.57,74,43,15.41
62,62,Sahibganj,IN,2021-07-23 21:35:21,25.2500,87.6500,80.53,89,100,16.31
87,87,Bethel,US,2021-07-23 21:35:29,41.3712,-73.4140,81.19,54,40,1.99


In [13]:
preferred_cities_df.count()

City_ID       90
City          90
Country       90
Date          90
Lat           90
Lng           90
Max Temp      90
Humidity      90
Cloudiness    90
Wind Speed    90
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
10,Guerrero Negro,MX,81.88,27.9769,-114.0611,
21,Kavaratti,IN,81.70,10.5669,72.6420,
27,Puerto Ayora,EC,82.36,-0.7393,-90.3518,
29,Victoria,HK,86.38,22.2855,114.1577,
32,Guajara,BR,85.05,-2.9667,-57.6667,
51,Fuerte Olimpo,PY,81.21,-21.0415,-57.8738,
54,Paracuru,BR,80.62,-3.4100,-39.0306,
59,Butaritari,KI,83.57,3.0707,172.7902,
62,Sahibganj,IN,80.53,25.2500,87.6500,
87,Bethel,US,81.19,41.3712,-73.4140,


In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
10,Guerrero Negro,MX,81.88,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
21,Kavaratti,IN,81.70,10.5669,72.6420,Hotels in Lakshadweep Islands
27,Puerto Ayora,EC,82.36,-0.7393,-90.3518,Finch Bay Galapagos Hotel
29,Victoria,HK,86.38,22.2855,114.1577,Mini Hotel Central
32,Guajara,BR,85.05,-2.9667,-57.6667,
...,...,...,...,...,...,...
533,North Bend,US,83.25,43.4065,-124.2243,Red Lion Hotel Coos Bay
543,Keti Bandar,PK,83.03,24.1447,67.4497,Haji laloo memon house
547,Portobelo,PA,82.81,9.5500,-79.6500,Scuba Portobelo
548,Itupiranga,BR,80.92,-5.1347,-49.3267,Hotel Panorama


In [18]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info 

['\n<dl>\n<dt>Hotel Name</dt><dd>Casa Laguna, Bed & Breakfast</dd>\n<dt>City</dt><dd>Guerrero Negro</dd>\n<dt>Country</dt><dd>MX</dd>\n<dt>Max Temp</dt><dd>81.88 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotels in Lakshadweep Islands</dd>\n<dt>City</dt><dd>Kavaratti</dd>\n<dt>Country</dt><dd>IN</dd>\n<dt>Max Temp</dt><dd>81.7 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Finch Bay Galapagos Hotel</dd>\n<dt>City</dt><dd>Puerto Ayora</dd>\n<dt>Country</dt><dd>EC</dd>\n<dt>Max Temp</dt><dd>82.36 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Mini Hotel Central</dd>\n<dt>City</dt><dd>Victoria</dd>\n<dt>Country</dt><dd>HK</dd>\n<dt>Max Temp</dt><dd>86.38 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Guajara</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Max Temp</dt><dd>85.05 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hospedaje DAMI</dd>\n<dt>City</dt><dd>Fuerte Olimpo</dd>\n<dt>Country</dt><dd>PY</dd>\n<dt>Max Temp</dt><dd>81.21 °F</dd>\n

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))